In [1]:
import os
import pickle

import pandas as pd
import mlflow
import uuid

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [2]:
year = 2023
month = 3
taxi_type = "yellow"
input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{taxi_type}/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'

In [3]:
RUN_ID = os.getenv('RUN_ID', '95182c88ec8040888af37b5f0259e733')
# RUN_ID = '95182c88ec8040888af37b5f0259e733'

In [4]:
#!mkdir output

In [5]:
#model

In [11]:
def generatre_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids
    
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)
   

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    n = len(df)
    df['ride_id'] = generatre_uuids(n)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [12]:
def load_model(run_id):
    logged_model = f's3://mlflows-artifacts-remote/1/{run_id}/artifacts/model'
    model = mlflow.pyfunc.load_model(logged_model)
    return model
    
def apply_model(input_file, run_id, output_file):
    df = read_dataframe(input_file)
    dict = prepare_dictionaries(df)
    model = load_model(run_id)
    y_pred = model.predict(dict)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['tpep_pickup_datetime'] = df['tpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = RUN_ID

    print(df_result.describe())
    
    df_result.to_parquet(output_file, index=False)

In [13]:
!mkdir output/yellow

mkdir: cannot create directory ‘output/yellow’: File exists


In [14]:
apply_model(input_file=input_file,run_id=RUN_ID,output_file=output_file) 

             tpep_pickup_datetime  actual_duration  predicted_duration  \
count                     3316216     3.316216e+06        3.316216e+06   
mean   2023-03-16 11:27:41.238746     1.499996e+01        1.364858e+01   
min           2002-12-31 23:10:19     1.000000e+00        1.542716e+00   
25%    2023-03-08 18:28:49.750000     7.483333e+00        7.256021e+00   
50%    2023-03-16 11:56:07.500000     1.211667e+01        1.090540e+01   
75%           2023-03-24 08:51:58     1.930000e+01        1.736589e+01   
max           2023-04-05 20:17:42     6.000000e+01        5.240148e+01   
std                           NaN     1.060465e+01        8.973342e+00   

               diff  
count  3.316216e+06  
mean   1.351382e+00  
min   -4.964534e+01  
25%   -1.622948e+00  
50%    5.140035e-01  
75%    3.684999e+00  
max    5.677213e+01  
std    5.673047e+00  


In [19]:
!ls output/green


green_tripdata_2021-02.parquet


In [20]:
!pwd

/home/ubuntu/mlopszoomcamp/04-deployment/batch
